In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1.0  YOLO To Detect Sections Of Music

In [ ]:
!pip install opencv-python ultralytics==8.0.117
!pip install albumentations==1.4.0

In [ ]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import torch
from datetime import datetime

now = datetime.now()
file_name_format = now.strftime("%Y-%m-%d_%H-%M-%S")

# Check if CUDA is available
print(torch.cuda.is_available())

model = YOLO('yolov8n.pt')

model.train(
    data="/content/drive/MyDrive/Sheet Music ML/sheet-music/dataset/data.yaml",
    epochs=30,
    imgsz=640,
    batch=2,
    project='/content/drive/MyDrive/Sheet Music ML/sheet-music/models/',
    name=file_name_format,
    device='cuda')

# Fine tune model on custom dataset
# model.train(
#     data="/content/drive/MyDrive/Sheet Music ML/sheet-music/dataset/data.yaml",
#     epochs=30,
#     imgsz=640,
#     batch=4,
#     project='/content/drive/MyDrive/Sheet Music ML/sheet-music/models/',
#     name=file_name_format)

True


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:511: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


# 2.0 Siamese Neural Network To Detect Line Changes

In [2]:
# load data

import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import torch

transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])  # Same as ImageNet
                                ])

path = "/content/drive/MyDrive/Sheet Music ML/sheet-music/siamese_dataset/"
folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]

dataset = []
images_untouched = []
labels = []

for folder in folders:
  images = [file for file in os.listdir(path + folder) if file.endswith(".png")]
  with open(path + folder + "/label.txt", 'r') as label_file:
    for line in label_file:
      label = int(line.strip())

  img_tensors = [transform(Image.open(os.path.join(path, folder, image))) for image in images]

  if len(img_tensors) == 2:
    dataset.append(img_tensors)
    images_untouched.append([Image.open(os.path.join(path, folder, image)) for image in images])
    labels.append(label)
  else:
    print(f"Corrupted folder {folder} does not contain 2 images, skipping")

from collections import Counter
label_counts = Counter(labels)
num_zeros = label_counts[0]
num_ones = label_counts[1]
print(f"Number of 0s: {num_zeros}")
print(f"Number of 1s: {num_ones}")

labels = torch.tensor(labels)


# for i in range(len(dataset)):
#   plt.imshow(dataset[i][0])
#   plt.show()
#   plt.imshow(dataset[i][1])
#   plt.show()
#   print(labels[i])


Number of 0s: 512
Number of 1s: 494


In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class SiameseNetwork(nn.Module):
  def __init__(self):
    super(SiameseNetwork, self).__init__()
    self.resnet = models.resnet18(pretrained=True)
    self.resnet.fc = nn.Identity() # Remove classification head

    self.fc1 = nn.Linear(512, 256)
    self.fc2 = nn.Linear(256, 64)

  def forward_one(self, x):
    x = self.resnet(x)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

  def forward(self, input1, input2):
    output1 = self.forward_one(input1)
    output2 = self.forward_one(input2)
    return output1, output2

class ContrastiveLoss(nn.Module):
  def __init__(self, margin=1.0):
    super(ContrastiveLoss, self).__init__()
    self.margin = margin

  def forward(self, output1, output2, label):
    euclidean_distance = F.pairwise_distance(output1, output2)
    loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                  (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
    return loss_contrastive


In [8]:
import torch.optim as optim
from datetime import datetime

model = SiameseNetwork()
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

num_epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
  for i in range(len(dataset)):
    img1 = dataset[i][0].to(device)
    img2 = dataset[i][1].to(device)
    label = labels[i].float().to(device)
    optimizer.zero_grad()

    output1, output2 = model(img1.unsqueeze(0), img2.unsqueeze(0))
    loss = criterion(output1, output2, label)
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch+1}/{num_epochs} completed.")
  print(f"Loss: {loss.item()}")

now = datetime.now()
file_name_format = now.strftime("%Y-%m-%d_%H-%M-%S")
path = '/content/drive/MyDrive/Sheet Music ML/sheet-music/models/siamese/'+file_name_format+'.pt'
print(f"Saving model to file {path}")

torch.save(model.state_dict(), path)

Epoch 1/30 completed.
Loss: 0.00024576165014877915
Epoch 2/30 completed.
Loss: 0.004225281532853842
Epoch 3/30 completed.
Loss: 0.007457312196493149
Epoch 4/30 completed.
Loss: 0.020294873043894768
Epoch 5/30 completed.
Loss: 0.1177394837141037
Epoch 6/30 completed.
Loss: 0.11159007996320724
Epoch 7/30 completed.
Loss: 0.007048579398542643
Epoch 8/30 completed.
Loss: 0.011517849750816822
Epoch 9/30 completed.
Loss: 0.07634278386831284
Epoch 10/30 completed.
Loss: 0.04363927245140076
Epoch 11/30 completed.
Loss: 0.0010042472276836634
Epoch 12/30 completed.
Loss: 0.01789710856974125
Epoch 13/30 completed.
Loss: 0.006334254983812571
Epoch 14/30 completed.
Loss: 0.034578897058963776
Epoch 15/30 completed.
Loss: 0.013516903854906559
Epoch 16/30 completed.
Loss: 0.00436874246224761
Epoch 17/30 completed.
Loss: 0.004754906520247459
Epoch 18/30 completed.
Loss: 0.0009877593256533146
Epoch 19/30 completed.
Loss: 0.007632074411958456
Epoch 20/30 completed.
Loss: 0.0010828913655132055
Epoch 21/30